In [3]:
import sys, os
sys.path.append('deep-learning-from-scratch')

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

%matplotlib inline

# 畳み込みNN

画像認識、音声認識に使われる。

今までのNNは全結合といい、隣接する層の全てのニューロンで結合があった。(Affineレイヤ)

CNNでは、

- 「covolutionレイヤ」
- 「活性化関数のレイヤ」
- 「poolingレイヤ」

という流れになる

出力に近い層では、これまでのAffineレイヤ＋ReLUが用いられ、
最後の出力層ではAffine+Softmaxという組み合わせがよく用いられる。

全結合の問題点は、データの形状が無視されてしまうこと。

### Conolution層(畳み込み層) 
ここでは形状が維持される。画像の場合は3次元データっとして入力を受取、次の層へ渡す。

 CNNでは、畳み込み層の入出力データを「特徴マップ」と呼ぶ。
 入力データを「入力特徴マップ」出力データを「出力特徴マップ」。
 
### padding
 
 データの端っこに固定データを置くことで、出力データのサイズを調整する
 
 
### ストライド

フィルタを適用する位置の間隔。 (いくつずつずらしていくか)

### プーリング層

縦横の空間を小さくする演算

#### Maxプーリング
区画の中で一番大きな価を採用する

#### プーリング層の特徴

- 学習するパラメータが無い
- チャンネル数は変化しない
- 微小な位置の変化にロバスト

## Convolution/Poolingレイヤの実装

(10, 1, 28, 28):　28 x 28　で1チャンネルのデータが10個あるということ

CNNではこのように4次元データを扱っていく






In [12]:
x = np.random.rand(3, 1, 5, 5)
x

array([[[[  8.01936398e-01,   2.38932919e-01,   5.33330968e-01,
            8.59232236e-01,   9.31647153e-01],
         [  3.18487999e-01,   7.75237934e-01,   9.85999465e-02,
            9.55828628e-01,   5.51337750e-01],
         [  9.28949661e-01,   5.35015357e-01,   5.52705007e-01,
            3.96487482e-01,   6.67635726e-01],
         [  3.72904590e-01,   9.55926636e-01,   8.73171904e-01,
            5.30996854e-02,   7.24388449e-01],
         [  2.83660739e-01,   8.56477809e-01,   2.10165412e-01,
            2.22770795e-01,   2.75942377e-01]]],


       [[[  7.50638528e-01,   1.93023507e-01,   9.74806351e-01,
            6.76417337e-01,   1.22456275e-01],
         [  8.04982432e-01,   7.40779482e-01,   7.78837038e-01,
            8.73085972e-01,   3.57306549e-01],
         [  7.12063053e-01,   8.18995367e-01,   4.99587257e-01,
            8.31199473e-01,   8.76681874e-01],
         [  8.54726009e-01,   1.26505465e-01,   4.06345400e-01,
            6.14133572e-01,   3.52424203e-01

In [11]:
x[0, 0] # もしくは　x[0][0]

array([[ 0.19207596,  0.08360162,  0.83104358,  0.1156216 ,  0.08232693],
       [ 0.75027477,  0.00384508,  0.08761624,  0.39792277,  0.25608106],
       [ 0.03915868,  0.80513211,  0.86443946,  0.5710307 ,  0.05639113],
       [ 0.90264263,  0.71361141,  0.57363867,  0.10336496,  0.3743136 ],
       [ 0.64801287,  0.04098883,  0.32367462,  0.91809708,  0.0178714 ]])

### im2colによる展開

In [13]:
from common.util import im2col

class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2 * self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2 * self.pad -FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.w.reshape(FN, -1).T # フィルタの展開

        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2) # カラムの順番を元の順番に戻す
        

In [14]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad 
    
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (H - self.pool_w) / self.stride)
        
        # 展開(1)
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h * self.pool_w)
        
        # 最大値(2)
        out = np.max(col, axis=1)
        
        # 整形(3)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        
        return out

<img src="screenshot/pooling_layer.png">

- 重みレイヤのサイズ分の区画を、1行に展開
- 行の中でpooling1列ののデータへ
- reshapeして出力データの形に変更する


## CNNの実装

「Convolution - ReLU - Pooling - Affine - ReLU - Affine - Softmax」のかたちでMNISTを解いていく

